# Store Setup

In [1]:
from RES.hdf5_handler import DataHandler
store="data/store/resources_BC.h5"
res_data=DataHandler(store)
cells=res_data.from_store('cells')
boundary=res_data.from_store('boundary')
solar_clusters=res_data.from_store('clusters/solar')
wind_clusters=res_data.from_store('clusters/wind')
wind_ts=res_data.from_store('timeseries/wind')
solar_ts=res_data.from_store('timeseries/solar')
solar_clusters_ts=res_data.from_store('timeseries/clusters/solar')
wind_clusters_ts=res_data.from_store('timeseries/clusters/wind')
di_wind=res_data.from_store('dissolved_indices/wind')
di_solar=res_data.from_store('dissolved_indices/solar')

FileNotFoundError: ``/local-scratch/localhome/mei3/eliasinul/work/BC_Combined_Modelling/models/RESource/data/store/resources_BC.h5`` does not exist

# Build Resources

In [ ]:
import RES.RESources as REsources

* Main module

In [ ]:
REsources.build_resources(provinces=['BC'],  #'BC','AB','SK','ON','NS','MB'
                          resource_types=['solar','wind'])

INFO:RES.utility:Directory 'data/downloaded_data/NREL/ATB/ATBe.parquet' found locally.


____________________________________________________________
     Initiating RESource Builder...
____________________________________________________________
 └> NREL_ATBProcessor initiated...
  └> Processing Annual Technology Baseline (ATB) data sourced from NREL...
  └> ATB cost datafile: ATBe.parquet loaded
  └> Extracting technology baseline costs...
  └─> Extracting Solar PV technology cost...
>> Data (GeoDataFrame/DataFrame) saved to data/store/resources_BC.h5 with key 'cost/atb/solar'
  └─> Extracting Wind Turbine technology cost...
>> Data (GeoDataFrame/DataFrame) saved to data/store/resources_BC.h5 with key 'cost/atb/wind'
  └─> Extracting BESS technology cost...


INFO:RES.AttributesParser:>> Loading GADM boundaries (Sub-provincial | level =2) for British Columbia  from local file data/processed_data/regions/gadm41_Canada_L2_BC.geojson.
INFO:atlite.data:Storing temporary files in /tmp/tmpy8h0yexg


>> Data (GeoDataFrame/DataFrame) saved to data/store/resources_BC.h5 with key 'cost/atb/bess'
 └> Snapshot for Resources: 2023-01-01 07:00:00 to 2024-01-01 06:00:00
__________________________________________________
 Initiating solar module for British Columbia...
 └> Preparing Grid Cells...

    >>> Memory management remarks:
    * After execution, all downloaded data is stored at cutout.path. By default, it is not loaded into memory, but into dask arrays. This keeps the memory consumption extremely low.
    * The data is accessible in cutout.data, which is an xarray.Dataset. Querying the cutout gives us some basic information on which data is contained in it.
    * For more operations related to cutout, check the tool docs @ https://atlite.readthedocs.io/en/master/examples/create_cutout.html#
        


INFO:RES.AttributesParser:>> Loading GADM boundaries (Sub-provincial | level =2) for British Columbia  from local file data/processed_data/regions/gadm41_Canada_L2_BC.geojson.
INFO:atlite.data:Storing temporary files in /tmp/tmpmt8g08ew
INFO:RES.AttributesParser:>> Loading global filters' rasters from GAEZ, trimmed to British Columbia
INFO:RES.AttributesParser:>> Raster file 'faocmb_2010.tif' already exists locally.
INFO:RES.AttributesParser:>> Raster file 'slpmed05.tif' already exists locally.
INFO:RES.AttributesParser:>> Raster file 'exclusion_2017.tif' already exists locally.
INFO:RES.AttributesParser:>> Loading GADM boundaries (Sub-provincial | level =2) for British Columbia  from local file data/processed_data/regions/gadm41_Canada_L2_BC.geojson.


...

Land availability (grid cells) map saved at vis/misc/land_availability_ERA5grid_British Columbia.png
Land availability map (excluder resolution) saved at vis/misc/land_availability_excluderResolution_British Columbia.png


INFO:RES.AttributesParser:>> Saving to the local store (as HDF5 file)
INFO:RES.AttributesParser:>> Loading GADM boundaries (Sub-provincial | level =2) for British Columbia  from local file data/processed_data/regions/gadm41_Canada_L2_BC.geojson.
INFO:atlite.data:Storing temporary files in /tmp/tmp73y3s1k1
INFO:RES.AttributesParser:>> Loading ERA5 Cutout
INFO:RES.AttributesParser:>> Loading GADM boundaries (Sub-provincial | level =2) for British Columbia  from local file data/processed_data/regions/gadm41_Canada_L2_BC.geojson.


>> Total ERA5 cells loaded : 2958 [each with .025 deg. (~30km) resolution ]
>> Updated 'cells' saved to data/store/resources_BC.h5 with key 'cells'
  └> Cells' capacity updated..
 └> Extracting ERA5 weather data...

    >>> Memory management remarks:
    * After execution, all downloaded data is stored at cutout.path. By default, it is not loaded into memory, but into dask arrays. This keeps the memory consumption extremely low.
    * The data is accessible in cutout.data, which is an xarray.Dataset. Querying the cutout gives us some basic information on which data is contained in it.
    * For more operations related to cutout, check the tool docs @ https://atlite.readthedocs.io/en/master/examples/create_cutout.html#
        
  └> GWA Cells not configured for solar.
  └─> Preparing Timeseries for the Cells...


INFO:atlite.data:Storing temporary files in /tmp/tmptv7a37e_
INFO:RES.AttributesParser:>> 2948 Grid Cells from Store Cutout
INFO:atlite.convert:Convert and aggregate 'pv'.



    >>> Memory management remarks:
    * After execution, all downloaded data is stored at cutout.path. By default, it is not loaded into memory, but into dask arrays. This keeps the memory consumption extremely low.
    * The data is accessible in cutout.data, which is an xarray.Dataset. Querying the cutout gives us some basic information on which data is contained in it.
    * For more operations related to cutout, check the tool docs @ https://atlite.readthedocs.io/en/master/examples/create_cutout.html#
        


# -------------- Test Starts--------------

In [ ]:
# # Iterate over provinces for both solar and wind resources
# resource_types = ['wind'] # , ,'solar'
# provinces=['BC']  #'AB','SK','ON','NS'
# for province_code in provinces:
#     for resource_type in resource_types:
#         required_args = {
#             "config_file_path": 'config/config.yaml',
#             "province_short_code": province_code,
#             "resource_type": resource_type
#         }
        
#         # Create an instance of Resources and execute the module
#         RES_module = REsources.RESources_builder(**required_args)

In [ ]:
# data1=RES_module.get_grid_cells()

In [ ]:
# data2=RES_module.get_cell_capacity()

In [ ]:
# data3=RES_module.extract_weather_data()

In [ ]:
# data4=RES_module.update_gwa_scaled_params()

In [ ]:
# data5=RES_module.get_CF_timeseries(cells)
# data6=RES_module.find_grid_nodes(cells,
#                                  use_pypsa_buses=False)

In [ ]:
# data7=RES_module.score_cells()
# data8=RES_module.get_clusters()

In [ ]:
# data9=RES_module.get_cluster_timeseries(solar_clusters,
#                                         di_solar,
#                                         solar_ts)

In [ ]:
# resource_clusters_wind,cluster_timeseries_wind=RES_module.select_top_sites(wind_clusters,
#                                                                 wind_clusters_ts,
#                                                                     resource_max_capacity=20)

In [ ]:
# resource_clusters_solar,cluster_timeseries_solar=RES_module.select_top_sites(solar_clusters,
#                                                                 solar_clusters_ts,
#                                                                     resource_max_capacity=10)



# -------------- Test Ends --------------

# Tests

def get_CRF(#self,
            r, 
            N):
    return (r * (1 + r) ** N) / ((1 + r) ** N - 1) if N > 0 else 0

def calculate_total_cost(#self, 
                             distance_to_grid_km: float, 
                             grid_connection_cost_per_km: float, 
                             tx_line_rebuild_cost: float, 
                             capex_tech: float,
                             capacity_MW:float) -> float:
        """
        Calculate the total cost, which includes the CAPEX and distance-based grid connection costs.
        Method: Simple Levelized Cost of Energy Calculation (https://www.nrel.gov/analysis/tech-lcoe-documentation.html)
        """
        # Calculate distance-based cost
        add_to_grid_cost = (distance_to_grid_km * grid_connection_cost_per_km / 1.60934) * (tx_line_rebuild_cost / 1.60934)  # Convert to miles as our costs are given in $/miles (USA study)

        # Total cost is CAPEX plus distance cost
        total_cost = capex_tech*capacity_MW + add_to_grid_cost  # in M$
        
        return total_cost

resource_type='wind'
interest_rate=0.03
N=cells[f'Operational_life_{resource_type}'].iloc[0]
CRF=get_CRF(interest_rate,N)
CF_column='wind_CF_mean'

def calculate_lcoe(row) -> float:
    """
    Calculate the potential LCOE score for each cell in the dataframe,
    reading cost parameters directly from the DataFrame.
    
    ## Args:
    - **row** : A single row of the DataFrame.m
    
    ## Returns:
    - **float** : The calculated LCOE value for the row.
    """
    # Calculate the total cost
    total_cost = calculate_total_cost(
        row['nearest_station_distance_km'],  # km
        row[f'grid_connection_cost_per_km_{resource_type}'],  # m$/km
        row[f'tx_line_rebuild_cost_{resource_type}'],  # m$/km
        row[f'capex_{resource_type}'],
        row[f'potential_capacity_{resource_type}']# MW
    ) # mW
    
    annual_energy = 8760  * row[CF_column] * row[f'potential_capacity_{resource_type}']# Total energy produced in a year
    if annual_energy == 0: # some cells have no potentials
        return float(99999)  # handle the error 
    else:
        # Calculate the LCOE

        lcoe = (total_cost * CRF / annual_energy)  # Avoid division by zero,  m$/MWh
    
        return lcoe *1E6 # Convert to $/MWh

cells_test=cells.copy()

cells_test[f'LCOE_{resource_type}'] = cells_test.apply(lambda row: calculate_lcoe(row), axis=1) # LCOE in $/MWh  # adopting NREL's method + some added costs
scored_dataframe = cells_test.sort_values(by=f'LCOE_{resource_type}', ascending=True).copy()  # Lower LCOE is better
scored_dataframe.head(10).LCOE_wind